-sandbox

<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning" style="width: 600px">
</div>

# MLflow Lab

In this lab we will explore the path to moving models to production with MLflow using the following steps:

1. Load in Airbnb dataset, and save both training dataset and test dataset as Delta tables
2. Train an MLlib linear regression model using all the listing features and tracking parameters, metrics artifacts and Delta table version to MLflow
3. Register this initial model and move it to staging using MLflow Model Registry
4. Add a new column, **`log_price`** to both our train and test table and update the corresponding Delta tables
5. Train a second MLlib linear regression model, this time using **`log_price`** as our target and training on all features, tracking to MLflow 
6. Compare the performance of the different runs by looking at the underlying data versions for both models
7. Move the better performing model to production in MLflow model registry

## ![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) In this lab you:<br>
- **Create Delta tables**
- **Track your MLlib model and Delta table version using MLflow**
- **Use MLflow model registry to version your models**

In [0]:
%pip install mlflow

Python interpreter will be restarted.
Collecting mlflow
 Using cached mlflow-1.28.0-py3-none-any.whl (17.0 MB)
Collecting cloudpickle<3
 Using cached cloudpickle-2.1.0-py3-none-any.whl (25 kB)
Collecting databricks-cli<1,>=0.8.7
 Using cached databricks_cli-0.17.2-py3-none-any.whl
Collecting click<9,>=7.0
 Using cached click-8.1.3-py3-none-any.whl (96 kB)
Requirement already satisfied: pytz<2023 in /databricks/python3/lib/python3.8/site-packages (from mlflow) (2020.5)
Collecting docker<6,>=4.0.0
 Using cached docker-5.0.3-py2.py3-none-any.whl (146 kB)
Collecting sqlparse<1,>=0.4.0
 Using cached sqlparse-0.4.2-py3-none-any.whl (42 kB)
Requirement already satisfied: entrypoints<1 in /databricks/python3/lib/python3.8/site-packages (from mlflow) (0.3)
Requirement already satisfied: requests<3,>=2.17.3 in /databricks/python3/lib/python3.8/site-packages (from mlflow) (2.25.1)
Requirement already satisfied: numpy<2 in /databricks/python3/lib/python3.8/site-packages (from mlflow) (1.20.1)
Requirement already satisfied: pandas<2 in /databricks/python3/lib/python3.8/site-packages (from mlflow) (1.2.4)
Requirement already satisfied: protobuf<5,>=3.12.0 in /databricks/python3/lib/python3.8/site-packages (from mlflow) (3.17.2)
Collecting prometheus-flask-exporter<1
 Using cached prometheus_flask_exporter-0.20.3-py3-none-any.whl (18 kB)
Collecting pyyaml<7,>=5.1
 Using cached PyYAML-6.0-cp38-cp38-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl (701 kB)
Collecting alembic<2
 Using cached alembic-1.8.1-py3-none-any.whl (209 kB)
Collecting gitpython<4,>=2.1.0
 Using cached GitPython-3.1.27-py3-none-any.whl (181 kB)
Requirement already satisfied: scipy<2 in /databricks/python3/lib/python3.8/site-packages (from mlflow) (1.6.2)
Collecting sqlalchemy<2,>=1.4.0
 Using cached SQLAlchemy-1.4.40-cp38-cp38-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.6 MB)
Collecting gunicorn<21
 Using cached gunicorn-20.1.0-py3-none-any.whl (79 kB)
Collecting importlib-metadata!=4.7.0,<5,>=3.7.0
 Using cached importlib_metadata-4.12.0-py3-none-any.whl (21 kB)
Requirement already satisfied: packaging<22 in /databricks/python3/lib/python3.8/site-packages (from mlflow) (20.9)
Collecting Flask<3
 Using cached Flask-2.2.2-py3-none-any.whl (101 kB)
Collecting querystring-parser<2
 Using cached querystring_parser-1.2.4-py2.py3-none-any.whl (7.9 kB)
Collecting importlib-resources
 Using cached importlib_resources-5.9.0-py3-none-any.whl (33 kB)
Collecting Mako
 Using cached Mako-1.2.1-py3-none-any.whl (78 kB)
Collecting oauthlib>=3.1.0
 Using cached oauthlib-3.2.0-py3-none-any.whl (151 kB)
Requirement already satisfied: six>=1.10.0 in /databricks/python3/lib/python3.8/site-packages (from databricks-cli<1,>=0.8.7->mlflow) (1.15.0)
Collecting pyjwt>=1.7.0
 Using cached PyJWT-2.4.0-py3-none-any.whl (18 kB)
Collecting tabulate>=0.7.7
 Using cached tabulate-0.8.10-py3-none-any.whl (29 kB)
Collecting websocket-client>=0.32.0
 Using cached websocket_client-1.4.0-py3-none-any.whl (54 kB)
Collecting itsdangerous>=2.0
 Using cached itsdangerous-2.1.2-py3-none-any.whl (15 kB)
Collecting Jinja2>=3.0
 Using cached Jinja2-3.1.2-py3-none-any.whl (133 kB)
Collecting Werkzeug>=2.2.2
 Using cached Werkzeug-2.2.2-py3-none-any.whl (232 kB)
Collecting gitdb<5,>=4.0.1
 Using cached gitdb-4.0.9-py3-none-any.whl (63 kB)
Collecting smmap<6,>=3.0.1
 Using cached smmap-5.0.0-py3-none-any.whl (24 kB)
Requirement already satisfied: setuptools>=3.0 in /usr/local/lib/python3.8/dist-packages (from gunicorn<21->mlflow) (52.0.0)
Collecting zipp>=0.5
 Using cached zipp-3.8.1-py3-none-any.whl (5.6 kB)
Requirement already satisfied: MarkupSafe>=2.0 in /databricks/python3/lib/python3.8/site-packages (from Jinja2>=3.0->Flask<3->mlflow) (2.0.1)
Requirement already satisfied: pyparsing>=2.0.2 in /databricks/python3/lib/python3.8/site-packages (from packaging<22->mlflow) (2.4.7)
Requirement already satisfied: python-dateutil>=2.7.3 in /d

In [0]:
%run "../Includes/Classroom-Setup"

Defining courseware-specific utility methods...

The source for this dataset is
wasbs://courseware@dbacademy.blob.core.windows.net/scalable-machine-learning-with-apache-spark/v01/

Your dataset directory is
dbfs:/user/manujkumar.joshi@celebaltech.com/dbacademy/machine_learning/datasets

Skipping install of existing dataset.

###  Step 1. Creating Delta Tables

#### Data versioning is an advantage of using Delta Lake, which preserves previous versions of datasets so that you can restore later.

Let's split our dataset into train and test datasets, and writing them out in Delta format. You can read more at the Delta Lake <a href="https://docs.delta.io/latest/index.html" target="_blank">documentation</a>.

In [0]:
file_path = f"{datasets_dir}/airbnb/sf-listings/sf-listings-2019-03-06-clean.delta/"
airbnb_df = spark.read.format("delta").load(file_path)

train_df, test_df = airbnb_df.randomSplit([.8, .2], seed=42)

In [0]:
train_delta_path = working_dir + "/train.delta"
test_delta_path = working_dir + "/test.delta"

# In case paths already exists
dbutils.fs.rm(train_delta_path, True)
dbutils.fs.rm(test_delta_path, True)

train_df.write.mode("overwrite").format("delta").save(train_delta_path)
test_df.write.mode("overwrite").format("delta").save(test_delta_path)

Let's now read in our train and test Delta tables, specifying that we want the first version of these tables. This <a href="https://databricks.com/blog/2019/02/04/introducing-delta-time-travel-for-large-scale-data-lakes.html" target="_blank">blog post</a> has a great example of how to read in a Delta table at a given version.

In [0]:
# TODO
data_version = 0
train_delta = spark.read.format('delta').option('versionAsOf',data_version).load(train_delta_path)
test_delta = spark.read.format('delta').option('versionAsOf',data_version).load(test_delta_path)

### Review Delta Table History
All the transactions for this table are stored within this table including the initial set of insertions, update, delete, merge, and inserts.

In [0]:
display(spark.sql(f"DESCRIBE HISTORY delta.`{train_delta_path}`"))

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
0,2022-08-26T06:25:39.000+0000,6997591375752473,manujkumar.joshi@celebaltech.com,WRITE,"Map(mode -> Overwrite, partitionBy -> [])",null,List(2051889157726538),0822-094520-n0irdqef,null,WriteSerializable,false,"Map(numFiles -> 4, numOutputRows -> 5786, numOutputBytes -> 208617)",null,Databricks-Runtime/10.4.x-scala2.12


By default Delta tables <a href="https://docs.databricks.com/delta/delta-batch.html#data-retention" target="_blank">keep a commit history of 30 days</a>. This retention period can be adjusted by setting **`delta.logRetentionDuration`**, which will determine how far back in time you can go. Note that setting this can result in storage costs to go up. 

<img src="https://files.training.databricks.com/images/icon_note_24.png"/> Be aware that versioning with Delta in this manner may not be feasible as a long term solution. The retention period of Delta tables can be increased, but with that comes additional costs to storage. Alternative methods of data versioning when training models and tracking to MLflow is to save copies of the datasets, either as an MLflow artifact (for a small dataset), or save to a separate distributed location and record the location of the underlying dataset as a tag in MLflow

### Step 2. Log initial run to MLflow

- **Let's first log a run to MLflow where we use all features**. We use the same approach with RFormula as before. 
- **This time however, let's also log both the version of our data and the data path to MLflow.**

In [0]:
# TODO
import mlflow
import mlflow.spark
from pyspark.ml.regression import LinearRegression
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import RFormula

with mlflow.start_run(run_name="lr_model") as run:
    # Log parameters
    mlflow.log_param("data_path", train_delta_path)  
    mlflow.log_param('data_version',data_version)
    mlflow.log_param('data_path',train_delta_path)
    # TODO: Log label: price-all-features
    # TODO: Log data_version: data_version


    # Create pipeline
    r_formula = RFormula(formula="price ~ .", featuresCol="features", labelCol="price", handleInvalid="skip")
    lr = LinearRegression(labelCol="price", featuresCol="features")
    pipeline = Pipeline(stages = [r_formula, lr])
    model = pipeline.fit(train_delta)

    # Log pipeline
    # TODO: Log model: model
    mlflow.spark.log_model(model,'model')

    # Create predictions and metrics
    pred_df = model.transform(test_delta)
    regression_evaluator = RegressionEvaluator(labelCol="price", predictionCol="prediction")
    rmse = regression_evaluator.setMetricName("rmse").evaluate(pred_df)
    r2 = regression_evaluator.setMetricName("r2").evaluate(pred_df)

    # Log metrics
    mlflow.log_metric('rmse',rmse)
    mlflow.log_metric('r2',r2)
    # TODO: Log RMSE
    # TODO: Log R2

    run_id = run.info.run_id

### Step 3. Register model and move to staging using MLflow Model Registry

We are happy with the performance of the above model and want to move it to staging. Let's create the model and register it to the MLflow model registry.

<img src="https://files.training.databricks.com/images/icon_note_24.png"/> Make sure the path to **`model_uri`** matches the subdirectory (the second argument to **`mlflow.log_model()`**) included above.

In [0]:
model_name = f"{cleaned_username}_mllib_lr"
model_uri = f"runs:/{run_id}/model"

model_details = mlflow.register_model(model_uri=model_uri, name=model_name)

Successfully registered model 'manujkumar_joshi_mllib_lr'.
2022/08/26 06:46:22 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: manujkumar_joshi_mllib_lr, version 1
Created version '1' of model 'manujkumar_joshi_mllib_lr'.

#### Transition model to staging.

In [0]:
from mlflow.tracking.client import MlflowClient

client = MlflowClient()

client.transition_model_version_stage(
    name=model_name,
    version=1,
    stage="Staging"
)

Out[14]: <ModelVersion: creation_timestamp=1661496382447, current_stage='Staging', description='', last_updated_timestamp=1661496444832, name='manujkumar_joshi_mllib_lr', run_id='88d9ddcaed724dbc9c67fab13752c356', run_link='', source='dbfs:/databricks/mlflow-tracking/2051889157726538/88d9ddcaed724dbc9c67fab13752c356/artifacts/model', status='READY', status_message='', tags={}, user_id='6997591375752473', version='1'>

In [0]:
# Define a utility method to wait until the model is ready
def wait_for_model(model_name, version, stage="None", status="READY", timeout=300):
    import time

    last_stage = "unknown"
    last_status = "unknown"

    for i in range(timeout):
        model_version_details = client.get_model_version(name=model_name, version=version)
        last_stage = str(model_version_details.current_stage)
        last_status = str(model_version_details.status)
        if last_status == str(status) and last_stage == str(stage):
            return

        time.sleep(1)

    raise Exception(f"The model {model_name} v{version} was not {status} after {timeout} seconds: {last_status}/{last_stage}")

In [0]:
# Force our notebook to block until the model is ready
wait_for_model(model_name, 1, stage="Staging")

#### Add a model description using <a href="https://mlflow.org/docs/latest/python_api/mlflow.tracking.html#mlflow.tracking.MlflowClient.update_registered_model" target="_blank">update_registered_model</a>.

In [0]:
# TODO
client.update_registered_model(
  name = model_details.name,
  description = 'forcasts prices of Airbnb based on inputs'
)

Out[17]: <RegisteredModel: creation_timestamp=1661496382105, description='forcasts prices of Airbnb based on inputs', last_updated_timestamp=1661496687413, latest_versions=[], name='manujkumar_joshi_mllib_lr', tags={}>

In [0]:
wait_for_model(model_details.name, 1, stage="Staging")

###  Step 4. Feature Engineering: Evolve Data Schema

- We now want to do some feature engineering with the aim of improving model performance; we can use Delta Lake to track older versions of the dataset. 

- We will add **`log_price`** as a new column and update our Delta table with it.

In [0]:
from pyspark.sql.functions import col, log, exp

# Create a new log_price column for both train and test datasets
train_new = train_delta.withColumn("log_price", log(col("price")))
test_new = test_delta.withColumn("log_price", log(col("price")))

Save the updated DataFrames to **`train_delta_path`** and **`test_delta_path`**, respectively, passing the **`mergeSchema`** option to safely evolve its schema. 

Take a look at this <a href="https://databricks.com/blog/2019/09/24/diving-into-delta-lake-schema-enforcement-evolution.html" target="_blank">blog</a> on Delta Lake for more information about **`mergeSchema`** (Schema Enforcement).

In [0]:
# TODO
train_new.write.option('mergeSchema','true').format('delta').mode('overwrite').save(train_delta_path)
test_new.write.option('mergeSchema','true').format('delta').mode('overwrite').save(test_delta_path)

Look at the difference between the original & modified schemas

In [0]:
set(train_new.schema.fields) ^ set(train_delta.schema.fields)

Out[21]: {StructField(log_price,DoubleType,true)}

Let's review the Delta history of our **`train_delta`** table and load in the most recent versions of our train and test Delta tables.

In [0]:
display(spark.sql(f"DESCRIBE HISTORY delta.`{train_delta_path}`"))

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
1,2022-08-26T07:00:36.000+0000,6997591375752473,manujkumar.joshi@celebaltech.com,WRITE,"Map(mode -> Overwrite, partitionBy -> [])",null,List(2051889157726538),0822-094520-n0irdqef,0,WriteSerializable,false,"Map(numFiles -> 2, numOutputRows -> 5786, numOutputBytes -> 191364)",null,Databricks-Runtime/10.4.x-scala2.12
0,2022-08-26T06:25:39.000+0000,6997591375752473,manujkumar.joshi@celebaltech.com,WRITE,"Map(mode -> Overwrite, partitionBy -> [])",null,List(2051889157726538),0822-094520-n0irdqef,null,WriteSerializable,false,"Map(numFiles -> 4, numOutputRows -> 5786, numOutputBytes -> 208617)",null,Databricks-Runtime/10.4.x-scala2.12


In [0]:
data_version = 1
train_delta_new = spark.read.format("delta").option("versionAsOf", data_version).load(train_delta_path)  
test_delta_new = spark.read.format("delta").option("versionAsOf", data_version).load(test_delta_path)

### Step 5. Use **`log_price`** as target and track run with MLflow

Retrain the model on the updated data and compare its performance to the original, logging results to MLflow.

In [0]:
with mlflow.start_run(run_name="lr_log_model") as run:
    # Log parameters
    mlflow.log_param("label", "log-price")
    mlflow.log_param("data_version", data_version)
    mlflow.log_param("data_path", train_delta_path)    

    # Create pipeline
    r_formula = RFormula(formula="log_price ~ . - price", featuresCol="features", labelCol="log_price", handleInvalid="skip")  
    lr = LinearRegression(labelCol="log_price", predictionCol="log_prediction")
    pipeline = Pipeline(stages = [r_formula, lr])
    pipeline_model = pipeline.fit(train_delta_new)

    # Log model and update the registered model
    mlflow.spark.log_model(
        spark_model=pipeline_model,
        artifact_path="log-model",
        registered_model_name=model_name
    )  

    # Create predictions and metrics
    pred_df = pipeline_model.transform(test_delta)
    exp_df = pred_df.withColumn("prediction", exp(col("log_prediction")))
    rmse = regression_evaluator.setMetricName("rmse").evaluate(exp_df)
    r2 = regression_evaluator.setMetricName("r2").evaluate(exp_df)

    # Log metrics
    mlflow.log_metric("rmse", rmse)
    mlflow.log_metric("r2", r2)  

    run_id = run.info.run_id

Registered model 'manujkumar_joshi_mllib_lr' already exists. Creating a new version of this model...
2022/08/26 07:07:23 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: manujkumar_joshi_mllib_lr, version 2
Created version '2' of model 'manujkumar_joshi_mllib_lr'.

### Step 6. Compare performance across runs by looking at Delta table versions 

Use MLflow's <a href="https://mlflow.org/docs/latest/python_api/mlflow.html#mlflow.search_runs" target="_blank">**`mlflow.search_runs`**</a> API to identify runs according to the version of data the run was trained on. Let's compare our runs according to our data versions.

Filter based on **`params.data_path`** and **`params.data_version`**.

In [0]:
# TODO
data_version = 0

mlflow.search_runs(filter_string=f"params.data_path='{train_delta_path}' and params.data_version='{data_version}'")

Out[26]:

,run_id,experiment_id,status,artifact_uri,start_time,end_time,metrics.rmse,metrics.r2,params.data_path,params.data_version,tags.mlflow.databricks.cluster.id,tags.mlflow.databricks.notebookRevisionID,tags.mlflow.user,tags.mlflow.databricks.workspaceID,tags.mlflow.databricks.workspaceURL,tags.mlflow.databricks.notebookPath,tags.mlflow.source.name,tags.mlflow.runName,tags.mlflow.databricks.notebookID,tags.mlflow.source.type,tags.mlflow.log-model.history,tags.mlflow.databricks.cluster.info,tags.mlflow.databricks.notebook.commandID,tags.mlflow.databricks.webappURL,tags.mlflow.databricks.cluster.libraries
0,88d9ddcaed724dbc9c67fab13752c356,2051889157726538,FINISHED,dbfs:/databricks/mlflow-tracking/2051889157726...,2022-08-26 06:42:07.962000+00:00,2022-08-26 06:44:27.834000+00:00,391.776823,0.129978,dbfs:/user/manujkumar.joshi@celebaltech.com/db...,0,0822-094520-n0irdqef,1661496267996,manujkumar.joshi@celebaltech.com,2542766428318180,adb-2542766428318180.0.azuredatabricks.net,/Users/manujkumar.joshi@celebaltech.com/Scalab...,/Users/manujkumar.joshi@celebaltech.com/Scalab...,lr_model,2051889157726538,NOTEBOOK,"[{""artifact_path"":""model"",""flavors"":{""spark"":{...","{""cluster_name"":""AjeetSingh Charan's Cluster"",...",2243176141899946539_7483794869853950968_ba4857...,https://centralindia.azuredatabricks.net,"{""installable"":[],""redacted"":[]}"


In [0]:
# TODO
data_version = 1

mlflow.search_runs(filter_string=f"params.data_path='{train_delta_path}' and params.data_version='{data_version}'")

Out[27]:

,run_id,experiment_id,status,artifact_uri,start_time,end_time,metrics.rmse,metrics.r2,params.data_path,params.label,params.data_version,tags.mlflow.databricks.cluster.id,tags.mlflow.databricks.notebookRevisionID,tags.mlflow.user,tags.mlflow.databricks.workspaceID,tags.mlflow.databricks.workspaceURL,tags.mlflow.databricks.notebookPath,tags.mlflow.source.name,tags.mlflow.runName,tags.mlflow.databricks.notebookID,tags.mlflow.source.type,tags.mlflow.log-model.history,tags.mlflow.databricks.cluster.info,tags.mlflow.databricks.notebook.commandID,tags.mlflow.databricks.webappURL,tags.mlflow.databricks.cluster.libraries
0,8a22e59b906440039f3da253e3688adc,2051889157726538,FINISHED,dbfs:/databricks/mlflow-tracking/2051889157726...,2022-08-26 07:05:07.325000+00:00,2022-08-26 07:07:50.344000+00:00,380.595604,0.178929,dbfs:/user/manujkumar.joshi@celebaltech.com/db...,log-price,1,0822-094520-n0irdqef,1661497670535,manujkumar.joshi@celebaltech.com,2542766428318180,adb-2542766428318180.0.azuredatabricks.net,/Users/manujkumar.joshi@celebaltech.com/Scalab...,/Users/manujkumar.joshi@celebaltech.com/Scalab...,lr_log_model,2051889157726538,NOTEBOOK,"[{""artifact_path"":""log-model"",""flavors"":{""spar...","{""cluster_name"":""AjeetSingh Charan's Cluster"",...",2243176141899946539_7576759484991374895_66753a...,https://centralindia.azuredatabricks.net,"{""installable"":[],""redacted"":[]}"


### Which version of the data produced the best model?

### Step 7. Move best performing model to production using MLflow model registry

Get the most recent model version and move it to production.

In [0]:
model_version_infos = client.search_model_versions(f"name = '{model_name}'")
new_model_version = max([model_version_info.version for model_version_info in model_version_infos])

In [0]:
client.update_model_version(
    name=model_name,
    version=new_model_version,
    description="This model version was built using a MLlib Linear Regression model with all features and log_price as predictor."
)

Out[29]: <ModelVersion: creation_timestamp=1661497643852, current_stage='None', description=('This model version was built using a MLlib Linear Regression model with all '
 'features and log_price as predictor.'), last_updated_timestamp=1661497979159, name='manujkumar_joshi_mllib_lr', run_id='8a22e59b906440039f3da253e3688adc', run_link='', source='dbfs:/databricks/mlflow-tracking/2051889157726538/8a22e59b906440039f3da253e3688adc/artifacts/log-model', status='READY', status_message='', tags={}, user_id='6997591375752473', version='2'>

In [0]:
model_version_details = client.get_model_version(name=model_name, version=new_model_version)
model_version_details.status

Out[30]: 'READY'

In [0]:
wait_for_model(model_name, new_model_version)

In [0]:
# TODO
# Move Model into Production
client.transition_model_version_stage(
  name = model_name,
  version= new_model_version,
  stage= 'Production'
)

Out[32]: <ModelVersion: creation_timestamp=1661497643852, current_stage='Production', description=('This model version was built using a MLlib Linear Regression model with all '
 'features and log_price as predictor.'), last_updated_timestamp=1661498076045, name='manujkumar_joshi_mllib_lr', run_id='8a22e59b906440039f3da253e3688adc', run_link='', source='dbfs:/databricks/mlflow-tracking/2051889157726538/8a22e59b906440039f3da253e3688adc/artifacts/log-model', status='READY', status_message='', tags={}, user_id='6997591375752473', version='2'>

In [0]:
wait_for_model(model_name, new_model_version, "Production")

Have a look at the MLflow model registry UI to check that your models have been successfully registered. You should see that version 1 of your model is now in staging, with version 2 in production.

### To finish the lab, let's clean up by archiving both model versions and deleting the whole model from the registry

In [0]:
client.transition_model_version_stage(
    name=model_name,
    version=1,
    stage="Archived"
)

Out[34]: <ModelVersion: creation_timestamp=1661496382447, current_stage='Archived', description='', last_updated_timestamp=1661498148192, name='manujkumar_joshi_mllib_lr', run_id='88d9ddcaed724dbc9c67fab13752c356', run_link='', source='dbfs:/databricks/mlflow-tracking/2051889157726538/88d9ddcaed724dbc9c67fab13752c356/artifacts/model', status='READY', status_message='', tags={}, user_id='6997591375752473', version='1'>

In [0]:
wait_for_model(model_name, 1, "Archived")

In [0]:
client.transition_model_version_stage(
    name=model_name,
    version=2,
    stage="Archived"
)

Out[36]: <ModelVersion: creation_timestamp=1661497643852, current_stage='Archived', description=('This model version was built using a MLlib Linear Regression model with all '
 'features and log_price as predictor.'), last_updated_timestamp=1661498156456, name='manujkumar_joshi_mllib_lr', run_id='8a22e59b906440039f3da253e3688adc', run_link='', source='dbfs:/databricks/mlflow-tracking/2051889157726538/8a22e59b906440039f3da253e3688adc/artifacts/log-model', status='READY', status_message='', tags={}, user_id='6997591375752473', version='2'>

In [0]:
wait_for_model(model_name, 2, "Archived")

In [0]:
client.delete_registered_model(model_name)

-sandbox
&copy; 2022 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="https://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a> | <a href="https://help.databricks.com/">Support</a>